# Web interface

> Module making it possible to run the conversion from slX-files from a web interface

In [ ]:
#| default_exp webgui

In [ ]:
#| hide
from nbdev.showdoc import *

## Imports and setting up of module

In [ ]:
#| export
import zipfile
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.jupyter import *
from fastlite import *
import fasthtml.components as fh
import httpx
from pathlib import Path

from sonar2csv_shape.convert import *

## Setting styling of web gui

When building the styling of the web application use the DaisyUI documentation from https://daisyui.com/llms.txt.

I want to apply a custom style using the color schemes of WDODelta.

**Setting Up Custom DaisyUI Themes in FastHTML:**

1. **Define your custom theme CSS** as a string with CSS variables wrapped in a `[data-theme="YourThemeName"]` selector

2. **Include the required headers:**
   ```python
   daisy_hdrs = (
       Link(rel="stylesheet", href="https://cdn.jsdelivr.net/npm/daisyui@5", type='text/css'),
       Script(src="https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4"),
       Link(href="https://cdn.jsdelivr.net/npm/daisyui@5/themes.css", rel='stylesheet', type='text/css'),
       Style(theme_css)
   )
   ```

3. **Apply your theme globally** using `htmlkw` when creating your FastHTML app:
   ```python
   app = FastHTML(hdrs=daisy_hdrs, htmlkw={'data-theme': 'WDODelta'})
   ```

**Key points:**
- Use `htmlkw` (not `bodykw`) to set `data-theme` on the `<html>` tag - this is where DaisyUI expects it
- The main daisyUI CSS (`https://cdn.jsdelivr.net/npm/daisyui@5`) provides component styles
- The separate `themes.css` file provides access to all built-in DaisyUI themes
- Your custom `theme_css` in a `Style()` tag defines your custom theme variables
- Don't use `/dist/full.min.css` - it doesn't work with daisyUI 5

WDODelta color theme I created with [daisyUI Theme Generator](https://daisyui.com/theme-generator/).

In [ ]:
#| export
theme_css = """
[data-theme="WDODelta"] {
  --color-base-100: oklch(100% 0 0);
  --color-base-200: oklch(98% 0 0);
  --color-base-300: oklch(95% 0 0);
  --color-base-content: oklch(21% 0.006 285.885);
  --color-primary: #075895;
  --color-primary-content: oklch(93% 0.034 272.788);
  --color-secondary: #00b0ea;
  --color-secondary-content: oklch(94% 0.028 342.258);
  --color-accent: #93c01f;
  --color-accent-content: oklch(38% 0.063 188.416);
  --color-neutral: oklch(14% 0.005 285.823);
  --color-neutral-content: oklch(92% 0.004 286.32);
  --color-info: #00b0ea;
  --color-info-content: oklch(29% 0.066 243.157);
  --color-success: #93c01f;
  --color-success-content: oklch(37% 0.077 168.94);
  --color-warning: #f29100;
  --color-warning-content: oklch(41% 0.112 45.904);
  --color-error: #d74116;
  --color-error-content: oklch(27% 0.105 12.094);
  --rounded-box: 0.5rem;
  --rounded-btn: 2rem;
  --rounded-badge: 2rem;
  --btn-text-case: uppercase;
}
"""

In [ ]:
#| export
daisy_hdrs = (
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css'),
    Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4'),
    Link(href='https://cdn.jsdelivr.net/npm/daisyui@5/themes.css', rel='stylesheet', type='text/css'),
    Style(theme_css),
)

In [ ]:
#| export
app = FastHTML(hdrs=daisy_hdrs, htmlkw={'data-theme': 'WDODelta'})
rt = app.route

In [ ]:
def get_preview(app):
    "Function to easily view part of the app in Jupyter Notebook or SolveIt"
    return partial(HTMX, app=app, host=None, port=None)

In [ ]:
p = get_preview(app)

::: {.callout-tip collapse="true"}
___
🤔 _Can you please explain to me what the partial function does, how it does that and what its purpose is?_

🤖Reply🤖
The partial function is a really useful tool from Python's functools module. Let me break it down for you:

What it does: partial takes a function and "freezes" some of its arguments, creating a new function with those arguments pre-filled.

How it works: When you call partial(func, arg1, arg2), it returns a new function that, when called, will automatically pass arg1 and arg2 to func, along with any additional arguments you provide later.

Simple example:

```python
def greet(greeting, name):
    return f"{greeting}, {name}!"
```

If you use partial(greet, "Hello"), you get a new function that already has "Hello" as the greeting, so you only need to provide the name later.

In your code, partial(HTMX, app=app, host=None, port=None) creates a new function where app, host, and port are already set. So when you call preview(...) later, those parameters are automatically included.

Does this make sense so far? Would you like me to show you a quick example of how partial works in practice?

:::

In [ ]:
@rt
def testing(): return Div(P('hi'), style='color:red')

`JupyUvi` is a function that can start the "Uvicorn" server in such a way that the response can be shown in both SolveIt and Jupyter Notebooks.

In [ ]:
srv = JupyUvi(app=app)

In [ ]:
p(testing)

NameError: name 'p' is not defined

## Create upload

See [FastHTML docs - quickstart](https://fastht.ml/docs/tutorials/quickstart_for_web_devs.html#single-file-uploads)

### Expanation of FastHTML fasttag `Titled()`

`Titled()` is a **convenience function** that creates multiple HTML elements at once (a `<title>` tag for the browser tab and an `<h1>` heading for the page). Since it's generating these elements internally, it doesn't expose styling parameters like `cls`.

In contrast, components like `P()` and `H1()` are **direct HTML element creators** - they map one-to-one to HTML tags. This gives you full control over their attributes, including `cls` for styling.

**The trade-off:** `Titled()` saves you time but limits customization, while using `Title()` + `H1()` separately gives you full control but requires more code.


### The difference between `Titled()` and `Title()` ft-components from `FastHTML` are: 

**`Titled(title, *content)`:**
- Sets the browser tab title
- Creates a visible h1 heading on the page
- Returns both together

**`Title(title)`:**
- Only sets the browser tab title
- Creates NO visible content on the page
- Must be combined with other elements to show anything

**The solution:** When you want a browser tab title but custom heading styling, return both:
```python
return Title("..."), YourVisibleContent(...)
```

## Create download

I asked the SolveIt Discord Buddy how to download files with FastHTML. This was Buddy's response.

For file downloads in FastHTML, create routes that return FileResponse with the generated files. Here's the pattern:

```python
from fasthtml.common import *
from starlette.responses import FileResponse

app = FastHTML()

@rt("/download/{filename}")
def get(filename:str):
    filepath = f"/app/data/{filename}"
    return FileResponse(filepath, filename=filename)

# In your upload handler, after processing:
@rt("/process")
async def post(file: UploadFile):
    # Your magic happens here, creates 6 files
    # Return links to download them
    links = [A(f"Download {i}", href=f"/download/file{i}.txt") for i in range(6)]
    return Div(*links)
```

For multiple files at once, you can create a zip file and serve that, or use HTMX to trigger multiple downloads. The key is using FileResponse from Starlette (included with FastHTML) which handles the browser download prompt automatically.

::: {.callout-tip collapse="true"}
___
🤔 _Lets write a page with a download button. When I hit that button it downloads a file from the /app/data folder. Use the information from the above note to know how to do this._

🤖Reply🤖
Here's a page with a download button:

We don't need to import startlette to get the FileResponse module, because that's already imported with FastHTML

```python
@rt
def download_file(filename:str):
    upload_dir = Path("/app/data/")
    filepath = upload_dir / filename
    return FileResponse(filepath, filename=filename)

@rt
def download():
    return Titled("File Download Demo",
        Article(
            H3("Download a file"),
            A("Download File", href="/download/test.txt", cls='btn')
        )
    )
You'll need to have a file called test.txt in your /app/data folder to test this. Want me to help you create a test file, or would you like to modify this to download a specific file you already have?
```
:::

A simple test for a download button using a file from the Python project directory, **without** variable injection (e.g. without flexible filename for download)

In [ ]:
dwnl_test = Path("../test/download_test.md")
dwnl_test.exists()

True

In [ ]:
@rt
def get_file():
    filepath = dwnl_test
    return FileResponse(filepath, filename=filepath.name)

@rt
def download():
    return Titled("File Download Demo",
        Article(
            H3("Download a file"),
            A("Download File", href="/get_file", cls='btn')
        )
    )

In [ ]:
p("download")

# FastHTML Best Practices


FastHTML applications are different to applications using FastAPI/react,
Django, etc. Don’t assume that FastHTML best practices are the same as
those for other frameworks. Best practices embody the fast.ai
philosophy: remove ceremony, leverage smart defaults, and write code
that’s both concise and clear. The following are some particular
opportunities that both humans and language models sometimes miss:

## Database Table Creation

**Before:**

``` python
todos = db.t.todos
if not todos.exists():
todos.create(id=int, task=str, completed=bool, created=str, pk='id')
```

**After:**

``` python
class Todo: id:int; task:str; completed:bool; created:str
todos = db.create(Todo)
```

FastLite’s `create()` is idempotent - it creates the table if needed and
returns the table object either way. Using a dataclass-style definition
is cleaner and more Pythonic. The `id` field is automatically the
primary key.

## Route Naming Conventions

**Before:**

``` python
@rt("/")
def get(): return Titled("Todo List", ...)

@rt("/add")
def post(task: str): ...
```

**After:**

``` python
@rt
def index(): return Titled("Todo List", ...) # Special name for "/"
@rt
def add(task: str): ... # Function name becomes route
```

Use `@rt` without arguments and let the function name define the route.
The special name `index` maps to `/`.

## Query Parameters over Path Parameters

**Before:**

``` python
@rt("/toggle/{todo_id}")
def post(todo_id: int): ...
# URL: /toggle/123
```

**After:**

``` python
@rt
def toggle(id: int): ...
# URL: /toggle?id=123
```

Query parameters are more idiomatic in FastHTML and avoid duplicating
param names in the path.

## Leverage Return Values

<div class="column-body-outset">

**Before:**

``` python
@rt
def add(task: str):
  new_todo = todos.insert(task=task, completed=False, created=datetime.now().isoformat())
  return todo_item(todos[new_todo])

@rt
def toggle(id: int):
  todo = todos[id]
  todos.update(completed=not todo.completed, id=id)
  return todo_item(todos[id])
```

**After:**

``` python
@rt
def add(task: str):
  return todo_item(todos.insert(task=task, completed=False, created=datetime.now().isoformat()))

@rt
def toggle(id: int):
  return todo_item(todos.update(completed=not todos[id].completed, id=id))
```

Both `insert()` and `update()` return the affected object, enabling
functional chaining.

</div>

## Use `.to()` for URL Generation

**Before:**

``` python
hx_post=f"/toggle?id={todo.id}"
```

**After:**

``` python
hx_post=toggle.to(id=todo.id)
```

The `.to()` method generates URLs with type safety and is
refactoring-friendly.

## PicoCSS comes free

**Before:**

``` python
style = Style("""
.todo-container { max-width: 600px; margin: 0 auto; padding: 20px; }
/* ... many more lines ... */
""")
```

**After:**

``` python
# Just use semantic HTML - Pico styles it automatically
Container(...), Article(...), Card(...), Group(...)
```

`fast_app()` includes PicoCSS by default. Use semantic HTML elements
that Pico styles automatically. Use MonsterUI (like shadcn, but for
FastHTML) for more complex UI needs.

## Smart Defaults

**Before:**

``` python
return Titled("Todo List", Container(...))

if __name__ == "__main__":
  serve()
```

**After:**

``` python
return Titled("Todo List", ...)  # Container is automatic

serve()  # No need for if __name__ guard
```

`Titled` already wraps content in a `Container`, and `serve()` handles
the main check internally.

## FastHTML Handles Iterables

**Before:**

``` python
Section(*[todo_item(todo) for todo in all_todos], id="todo-list")
```

**After:**

``` python
Section(map(todo_item, all_todos), id="todo-list")
```

FastHTML components accept iterables directly - no need to unpack with
`*`.

## Functional Patterns

List comprehensions are great, but `map()` is often cleaner for simple
transformations, especially when combined with FastHTML’s iterable
handling.

## Minimal Code

**Before:**

``` python
@rt
def delete(id: int):
  # Delete from database
  todos.delete(id)
  # Return empty response
  return ""
```

**After:**

``` python
@rt
def delete(id: int): todos.delete(id)
```

- Skip comments when code is self-documenting
- Don’t return empty strings - `None` is returned by default
- Use a single line for a single idea.

## Use POST for All Mutations

**Before:**

``` python
hx_delete=f"/delete?id={todo.id}"
```

**After:**

``` python
hx_post=delete.to(id=todo.id)
```

FastHTML routes handle only GET and POST by default. Using only these
two verbs is more idiomatic and simpler.

## Modern HTMX Event Syntax

**Before:**

``` python
hx_on="htmx:afterRequest: this.reset()"
```

**After:**

``` python
hx_on__after_request="this.reset()"
```

This works because:

- `hx-on="event: code"` is deprecated; `hx-on-event="code"` is preferred
- FastHTML converts `_` to `-` (so `hx_on__after_request` becomes
  `hx-on--after-request`)
- `::` in HTMX can be used as a shortcut for `:htmx:`.
- HTMX natively accepts `-` instead of `:` (so `-htmx-` works like
  `:htmx:`)
- HTMX accepts e.g `after-request` as an alternative to camelCase
  `afterRequest`
Highlight connections
3 connections found
Actions


What is the most FastHTML way to create the download routes? I now have this. But that doesn't seem to work.

```python
@rt
def get_file(fn: str):
    # filepath = "/app/data/solveit_settings.json" # filename
    pn = Path("/home/jelle/code_data/")
    filepath = pn/fn
    return FileResponse(filepath, filename=fn)

@rt("/download_page")
def get():
    fn = "dwnltest.dbf"
    return Titled("File Download Demo",
        Article(
            H3("Download a file"),
            A("Download File", href=f"/get_file/{fn}", cls='btn')
        )
    )
```

##### 🤖Reply🤖<!-- SOLVEIT_SEPARATOR_7f3a9b2c -->

Looking at your code and the FastHTML best practices, there are a couple of issues:

1. You're using a path parameter (`/get_file/{fn}`) but FastHTML prefers query parameters
2. The `A` tag's `href` should use `.to()` for URL generation

Here's the FastHTML way:

```python
@rt
def get_file(fn: str):
    pn = Path("/home/jelle/code_data/")
    filepath = pn/fn
    return FileResponse(filepath, filename=fn)

@rt
def download_page():
    fn = "dwnltest.dbf"
    return Titled("File Download Demo",
        Article(
            H3("Download a file"),
            A("Download File", href=get_file.to(fn=fn), cls='btn')
        )
    )
```

The key changes:
- Use `@rt` without the path - let the function name define the route
- Use `get_file.to(fn=fn)` instead of manually constructing the URL
- This creates `/get_file?fn=dwnltest.dbf` as a query parameter

Does this work for you?

A simple test for a download button using a file from the Python project directory, **with** variable injection (e.g. with flexible filename for download)

In [ ]:
@rt
def get_file(fn: str):
    upload_dir = Path("../test")
    filepath = upload_dir/fn
    return FileResponse(filepath, filename=fn)

In [ ]:
@rt
def download_page():
    fn = "download_test.md"
    return Titled("File Download Demo",
        Article(
            H3("Download a file"),
            A("Download File", href=get_file.to(fn=fn), cls='btn')
        )
    )

In [ ]:
p(download_page)

## Web application upload, convert and download

### `sonarhome()` Documentation

**Purpose:** Creates the main landing page for the Sonar File Converter web application, allowing users to upload `.sl2`/`.sl3` sonar files and receive converted shapefiles, CSV, and geopackage files as a downloadable zip.

**How it works:**

1. **Layout & Branding:** Uses a full-height background image (WDODelta imagery) with a centered title and form, styled with Tailwind CSS utility classes.

2. **File Upload Form:** Contains a file input and submit button that:
   - Posts to the `upload` route via HTMX (`hx_post=upload`)
   - Displays results in `#result` div (`hx_target="#result"`)
   - Shows a loading spinner during processing (`hx_indicator="#spinner"`)
   - Disables the button while uploading (`hx_disabled_elt="button"`)

3. **Uses `Title()` instead of `Titled()`:** This gives full control over styling the visible heading with custom Tailwind classes (`text-4xl font-bold`), which `Titled()` wouldn't allow since it auto-generates an `<h1>`.

**Why this approach?**

- **HTMX over full page reload:** Keeps the user on the same page; only the result area updates—better UX for a file conversion workflow.
- **DaisyUI + Tailwind:** Provides the spinner, button styles, and file input styling with minimal custom CSS, matching the WDODelta theme defined earlier.
- **Separation of concerns:** The upload logic lives in a separate `upload()` route, keeping `sonarhome()` focused purely on presentation.

**`index()` route:** Redirects `/` to `/sonarhome` so the app works both in production (where users expect `/` to work) and in SolveIt (where you can preview `/sonarhome`).

In [ ]:
#| export
@rt
def sonarhome():
    return Div(
        Title("Sonar File Converter"),
        Div(cls="min-h-screen bg-cover bg-center",
            style="background-image: url('https://cuatro.sim-cdn.nl/wdodelta/uploads/styles/large_5x2_2560x1024/media/foto_-_stuw_en_duiker_bij_lambert_van_der_linde_albertzoon_weg_3_0.jpg?h=e600bad3&cb=Y-KMqDmb')"
            )(H1("Sonar File Converter", cls="text-4xl font-bold text-center p-8"),
            Article(cls="flex flex-col items-center justify-center ")(
                Form(hx_post=upload, hx_target="#result", hx_indicator="#spinner", hx_disabled_elt="button", cls="flex gap-2")(
                    Input(type="file", name="file", cls="file-input file-input-bordered"),
                    Button("Upload and Convert", type="submit", cls='btn btn-primary'),
                    Span(cls="loading loading-spinner loading-lg htmx-indicator", id="spinner")
                ),
                Div(id="result")
            )
        )
    )


### `upload()` Documentation

**Purpose:** Handles sonar file uploads, converts them to GIS formats (shapefile, CSV, geopackage) using `process_sonar_file()`, packages all outputs into a zip, and provides a download link while cleaning up temporary files.

**How it works:**

1. **Receives file:** Reads the uploaded file buffer and saves it to `/app/data`
2. **Converts:** Calls `process_sonar_file()` to generate `.shp`, `.shx`, `.dbf`, `.prj`, `.cpg`, `.csv`, and `.gpkg` files
3. **Packages:** Creates a zip file containing all generated outputs, deleting originals as they're added
4. **Cleanup:** Removes the uploaded sonar file immediately
5. **Returns:** A `Div` with a success message and download link that triggers final zip cleanup after download

**Why this approach?**

- **Async function:** Handles `await file.read()` properly for non-blocking file uploads
- **Zip creation inline:** Uses Python's `zipfile` to bundle multiple outputs—simpler than serving 6+ separate files
- **Immediate cleanup:** Deletes source files as they're zipped to minimize disk usage
- **HTMX cleanup trigger:** `hx_on__after_request` fires a POST to `/cleanup` after download completes, automatically removing the zip without requiring a separate user action
- **Query parameters:** Uses `get_file.to(fn=...)` following FastHTML best practices for URL generation

In [ ]:
#| export
@rt
async def upload(file: UploadFile):
    upload_dir = Path("/app/data")
    filebuffer = await file.read()
    print(file.filename)
    input_path = upload_dir / file.filename
    input_path.write_bytes(filebuffer)
    
    process_sonar_file(input_path, upload_dir)
    
    stem = Path(input_path).stem
    zip_path = upload_dir / f"{stem}.zip"
    with zipfile.ZipFile(zip_path, 'w') as zf:
        for ext in ['.shp', '.shx', '.dbf', '.prj', '.cpg', '.csv', '.gpkg']:
            fp = upload_dir / f"{stem}{ext}"
            if fp.exists(): zf.write(fp, fp.name); fp.unlink()
    
    input_path.unlink()
    
    return Div(
        P(f"Conversion complete!"),
        A("Download Results", href=get_file.to(fn=zip_path.name), cls='btn btn-success', 
          hx_on__after_request=f"fetch('/cleanup?fn={zip_path.name}', {{method: 'POST'}})")
    )

### `get_file()` Documentation

**Purpose:** Serves files from `/app/data` for download to the user's browser.

**How it works:**

1. **Receives filename:** Takes `fn` as a query parameter (e.g., `/get_file?fn=results.zip`)
2. **Constructs path:** Combines `/app/data` with the filename
3. **Returns FileResponse:** Starlette's `FileResponse` handles the download, setting appropriate headers so the browser prompts to save the file

**Why this approach?**

- **Query parameters over path parameters:** Following FastHTML best practices—simpler and more idiomatic than `/get_file/{fn}`
- **FileResponse handles complexity:** Automatically manages content-type headers, streaming large files, and browser download prompts
- **Minimal code:** Single-purpose function that does one thing clearly
- **Works with `.to()` method:** Enables type-safe URL generation like `get_file.to(fn=zip_path.name)` in other routes

In [ ]:
#| export
@rt
def get_file(fn: str):
    upload_dir = Path("/app/data")
    filepath = upload_dir/fn
    return FileResponse(filepath, filename=fn)

### `cleanup()` Documentation

**Purpose:** Deletes temporary zip files from `/app/data` after the user has downloaded them.

**How it works:**

1. **Receives filename:** Takes `fn` as a query parameter (e.g., `/cleanup?fn=results.zip`)
2. **Constructs path:** Combines `/app/data` with the filename
3. **Safe deletion:** Checks if file exists before attempting to delete it with `unlink()`

**Why this approach?**

- **Triggered automatically:** Called via HTMX's `hx_on__after_request` event in the `upload()` function—runs after the download completes without user interaction
- **Query parameters:** Follows FastHTML best practices for passing data
- **Minimal code:** Single responsibility—just delete the file
- **No return value needed:** FastHTML returns `None` by default, which is appropriate for cleanup operations
- **Existence check:** Prevents errors if the file was already deleted or never created

This completes the workflow: upload → convert → download → automatic cleanup.

In [ ]:
#| export
@rt
def cleanup(fn: str): 
    upload_dir = Path("/app/data")
    fp = upload_dir / fn
    if fp.exists(): fp.unlink()

In [ ]:
p(sonarhome)

## Adding option to start the webserver locally with `serve()`

First make sure the `index` (e.g. "/" or root) page is redirected to the homepage of this app: "sonarhome".

In [ ]:
#| export
@rt
def index(): return RedirectResponse('/sonarhome')

Next, add a function we can start from the commandline to start the server and the client for the  webapplication.

In [ ]:
#| export
def main():
    """Start the web server locally for the sonar file converter"""
    serve()

By adding this line to `pyproject.toml` we can make this function available on the commandline.

```./pyproject.toml
...
[project.scripts]
sonar2csv_webgui = "sonar2csv_shape:webgui_main"
...
```

And expose this function by adding tot the `__init__.py` file in the folder `./sonar2csv_shape` (e.g. the folder with the project python files).

```./sonar2csv_shape/__init__.py
...
from .webgui import main as webgui_main

__all__ = ['wegbui_main', ...]
```

Then we can run from the root of this project.

```bash
uv sync
uv run sonar2csv_webgui
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()